# Store Sales - Time Series Forecasting

Use machine learning to predict grocery sales [dataset - link](https://www.kaggle.com/competitions/store-sales-time-series-forecasting)

The training data, comprising time series of features `store_nbr`, `family`, and `onpromotion` as well as the target `sales`.
- `store_nbr` identifies the store at which the products are sold.
- `family` identifies the type of product sold.
- `sales` gives the total sales for a product family at a particular store at a given date. Fractional values are possible since products can be sold in fractional units (1.5 kg of cheese, for instance, as opposed to 1 bag of chips).
- `onpromotion` gives the total number of items in a product family that were being promoted at a store at a given date.


In [60]:
# Import of packages
import duckdb
import warnings
import numpy as np
import pandas as pd
import plotly.express as px
import matplotlib.pyplot as plt
warnings.filterwarnings("ignore")
from statsforecast import StatsForecast
from statsforecast.models import AutoETS

In [61]:
# Reading .csv files via DuckDB will facilitate and optimize time series searching and extraction
con = duckdb.connect(database=':memory:', read_only=False)
con.execute("CREATE TABLE train AS SELECT * FROM read_csv('./train.csv')")

In [62]:
# Variables of interest
labelx, labely= 'date','sales'

Analyzing the training data, it is possible to see that we have 54 stores and each store has 33 products or types of products sold by them, as sales are daily
and even when there are no sales, the series is registered, we have at least 1782 distinct temporary series, as the records go from 01/01/2013 to 08/15/2017, we would have
1782*1687(days), the training file would have around 3006234 which is close to the real value of 3000888 lines in the set.

In [63]:
data = con.execute('SELECT * FROM train')
dataframe = data.fetch_df()
print(f"Number of records: {len(dataframe[labelx])}")

Number of records: 3000888


In [64]:
# This function will extract the individual series using the store and product type columns
def extract_series(col1, col2):
    sql_query = f"SELECT * FROM train WHERE store_nbr = '{col1}' AND family='{col2}'"
    serie_tmp =  con.execute(sql_query)
    df_tmp = serie_tmp.fetch_df()
    df_tmp['date'] = pd.to_datetime(df_tmp['date'])
    return df_tmp.sort_values(by='date')

In [65]:
# Example of the store 11 series and the SEAFOOD product
serie =  extract_series(11, 'SEAFOOD')
serie

,id,date,store_nbr,family,sales,onpromotion
0,98,2013-01-01,11,SEAFOOD,0.0,0
1,1880,2013-01-02,11,SEAFOOD,5.0,0
2,3662,2013-01-03,11,SEAFOOD,12.0,0
3,5444,2013-01-04,11,SEAFOOD,2.0,0
4,7226,2013-01-05,11,SEAFOOD,3.0,0
...,...,...,...,...,...,...
1679,2992076,2017-08-11,11,SEAFOOD,10.0,0
1680,2993858,2017-08-12,11,SEAFOOD,9.0,3
1681,2995640,2017-08-13,11,SEAFOOD,10.0,0
1682,2997422,2017-08-14,11,SEAFOOD,3.0,0


In [66]:
# This function extracts the series from month 08 of each available year, uses the part from month 08 to 15 to model the behavior of the series and returns a graph if the graph argument is true, and returns a series combined by the average of the last year and modeling as prediction
def predict_plot_series_year(data, plot=False):
    start_date_2013 = pd.to_datetime('2013-08-01')
    end_date_2013  = pd.to_datetime('2013-08-31')
    start_date_2014 = pd.to_datetime('2014-08-01')
    end_date_2014  = pd.to_datetime('2014-08-31')
    start_date_2015 = pd.to_datetime('2015-08-01')
    end_date_2015  = pd.to_datetime('2015-08-31')
    start_date_2016 = pd.to_datetime('2016-08-01')
    end_date_2016  = pd.to_datetime('2016-08-31')
    start_date_2017d = pd.to_datetime('2017-05-01')
    start_date_2017 = pd.to_datetime('2017-08-01')
    end_date_2017  = pd.to_datetime('2017-08-15')

    serie_2013 =  data[(data['date'].between(start_date_2013, end_date_2013))]['sales']
    serie_2014 =  data[(data['date'].between(start_date_2014, end_date_2014))]['sales']
    serie_2015 =  data[(data['date'].between(start_date_2015, end_date_2015))]['sales']
    serie_2016 =  data[(data['date'].between(start_date_2016, end_date_2016))]['sales']
    serie_2017 =  data[(data['date'].between(start_date_2017, end_date_2017))]['sales']
    serie_2017D =  data[(data['date'].between(start_date_2017d, end_date_2017))]
    

    train = pd.DataFrame({'ds': serie_2017D['date'], 'y': serie_2017D['sales'], 'unique_id': serie_2017D['family']})
    model = StatsForecast(models=[AutoETS(season_length=7)], freq='D', n_jobs=-1)
    model.fit(train)
    predictions = model.predict(h=16)

    zeros_2017 = pd.Series([float(0)] * 16)
    nan_values = pd.Series([float('NaN')] * 16)
    serie_2017p = pd.concat([serie_2017, pd.Series(predictions['AutoETS'].values)])
    serie_2017 = pd.concat([serie_2017, nan_values])
    Mean = serie_2017p.values
    if plot:
        index = [i for i in range(1, len(serie_2013) + 1)] 
        serie = pd.DataFrame({
        'Index': index,
        'Série 2016': serie_2016.values,
        'serie ETS' : serie_2017p.values,
        'Série 2017': serie_2017.values,
        'ETS + 2016': Mean
        })

        fig = px.line(serie, x='Index', y=list(serie.drop('Index', axis=1).columns))
        fig.show()
    
    return np.round(Mean[-16:], 3)

In [67]:
predict_plot_series_year(serie, plot=True)

array([ 9.999,  7.493,  6.122, 13.456, 16.547,  9.04 ,  9.279,  9.999,
        7.493,  6.122, 13.456, 16.547,  9.04 ,  9.279,  9.999,  7.493])

The test data ranges from 16/08/2017 to 31/08/2017, that is, the function `predict_plot_series_year` already returns the forecast based on the average

In [68]:
serie

,id,date,store_nbr,family,sales,onpromotion
0,98,2013-01-01,11,SEAFOOD,0.0,0
1,1880,2013-01-02,11,SEAFOOD,5.0,0
2,3662,2013-01-03,11,SEAFOOD,12.0,0
3,5444,2013-01-04,11,SEAFOOD,2.0,0
4,7226,2013-01-05,11,SEAFOOD,3.0,0
...,...,...,...,...,...,...
1679,2992076,2017-08-11,11,SEAFOOD,10.0,0
1680,2993858,2017-08-12,11,SEAFOOD,9.0,3
1681,2995640,2017-08-13,11,SEAFOOD,10.0,0
1682,2997422,2017-08-14,11,SEAFOOD,3.0,0


In [69]:
train = pd.DataFrame({'ds': serie['date'], 'y': serie['sales'], 'unique_id': serie['family']})

In [70]:
test = pd.read_csv('./test.csv')
dataset = test.copy()

In [71]:
def pred_series(loja, family):
    serie_tmp = extract_series(loja, family)
    id_tmp = dataset[(dataset['store_nbr']==loja) & (dataset['family']==family)]['id']
    predict = predict_plot_series_year(serie_tmp)
    result_df = pd.DataFrame({'id': id_tmp, 'sales': predict})
    
    return result_df
    

In [72]:
stores = test['store_nbr'].unique()
products = test['family'].unique()

In [73]:
predict = pd.DataFrame({'id': [], 'sales':[]})
for store in stores:
    for product in products:
        tmp = pred_series(store, product)
        predict=pd.concat([predict, tmp], axis=0)

In [74]:
predict['id'] = predict['id'].astype(int)

In [75]:
predict

,id,sales
0,3000888,4.131
1782,3002670,4.032
3564,3004452,6.722
5346,3006234,5.348
7128,3008016,2.179
...,...,...
21383,3022271,25.657
23165,3024053,17.754
24947,3025835,15.353
26729,3027617,14.660


In [76]:
predict = predict.sort_values(by='id')

In [77]:
predict.to_csv('./submission.csv', index=False)